# Sports Book: Over/Under Number of Points

## By Cup & By Map & By Team

This notebook uses the Game Outcomes Table (SQL) to compute the number of points, by cup and by map and by team. It uses the empirical CDF to compute this.

This data is used to create a new table (with a corresponding data class given below) that stores expected number of points by cup and by map and by team.

In [1]:
import os, sys, subprocess, json, time
import statistics
from pprint import pprint

In [2]:
from sqlalchemy import Column, Integer, String, Boolean, Table
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy import func, select, column

In [3]:
# SQL copypasta

class Base(DeclarativeBase):
    pass
    
class GameResult(Base):
    __tablename__ = "sportsbook_game_result"
    gameid = Column("gameid", String, primary_key=True)
    cup = Column("cup", String)
    season = Column("season", Integer)
    day = Column("day", Integer)
    mapName = Column("mapName", String)
    winnerAbbr = Column("winnerAbbr", String)
    winnerPoints = Column("winnerPoints", Integer)
    loserAbbr = Column("loserAbbr", String)
    loserPoints = Column("loserPoints", Integer)
    isPostseason = Column("isPostseason", Boolean)
    series = Column("series", String)
    generations = Column("generations", Integer)

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

fpath = os.path.join(os.environ['HOME'], 'tmp', 'sqlite', 'mydb.db')
engine = create_engine(f"sqlite:///{fpath}")
Session = sessionmaker(engine)
session = Session()

In [5]:
class MapPoints(Base):
    __tablename__ = "sportsbook_stats_points"
    id = Column("id", Integer, primary_key=True)
    cup = Column("cup", String)
    mapName = Column("mapName", String)
    # If teamAbbr is empty, it means this is the median for all games on this map
    teamAbbr = Column("teamAbbr", String)
    # This is the median number of points scored by this team on this map
    teamPoints = Column("teamPoints", Integer)

In [6]:
import time
time.sleep(2)

In [7]:
# Drop all prior results and re-calculate
session.query(MapPoints).delete()

0

In [8]:
if session.query(MapPoints).count() == 0:
    print(f"Confirmed that table {MapPoints.__tablename__} is empty")

Confirmed that table sportsbook_stats_points is empty


In [9]:
def add_map_points(this_cup, this_mapName, this_teamAbbr):
    # need to add query to get unique team abbrs for this cup from game outcomes
    # then, select WHERE where cup is cup, mapname is mapname, team is team
    query1 = select(GameResult.winnerPoints)\
        .where(GameResult.cup==this_cup)\
        .where(GameResult.mapName==this_mapName)\
        .where(GameResult.winnerAbbr==this_teamAbbr)
    
    query2 = select(GameResult.loserPoints)\
        .where(GameResult.cup==this_cup)\
        .where(GameResult.mapName==this_mapName)\
        .where(GameResult.loserAbbr==this_teamAbbr)
    
    res1 = session.execute(query1)
    res2 = session.execute(query2)
    
    values = [j[0] for j in res1] + [k[0] for k in res2]
    
    med = int(statistics.median(values))
    
    # Stash median value in the Map Generations Table
    g = MapPoints(
        cup = this_cup,
        mapName = this_mapName,
        teamAbbr = this_teamAbbr,
        teamPoints = med
    )
    session.add(g)

def add_aggregate_map_points(this_cup, this_mapName):
    query1 = select(GameResult.winnerPoints)\
        .where(GameResult.cup==this_cup)\
        .where(GameResult.mapName==this_mapName)
    
    query2 = select(GameResult.loserPoints)\
        .where(GameResult.cup==this_cup)\
        .where(GameResult.mapName==this_mapName)
    
    res1 = session.execute(query1)
    res2 = session.execute(query2)

    values = [j[0] for j in res1] + [k[0] for k in res2]

    med = int(statistics.median(values))
    
    g = MapPoints(
        cup = this_cup,
        mapName = this_mapName,
        teamAbbr = "",
        teamPoints = med
    )
    session.add(g)

In [10]:
# Get all the combinations of cup-mapName-teamAbbr, then iterate over each one
stmt1 = select(GameResult.cup, GameResult.mapName, GameResult.winnerAbbr).distinct()\
     .order_by(GameResult.cup, GameResult.mapName, GameResult.winnerAbbr)

stmt2 = select(GameResult.cup, GameResult.mapName, GameResult.loserAbbr).distinct()\
     .order_by(GameResult.cup, GameResult.mapName, GameResult.loserAbbr)

for stmt in [stmt1, stmt2]:
    for irow, row in enumerate(session.execute(stmt)):
        this_cup = row[0]
        this_mapName = row[1]
        this_teamAbbr = row[2]
        # Pass to the function that determines median number of generations for this cup-map combo, and inserts it
        add_map_points(this_cup, this_mapName, this_teamAbbr)

In [11]:
# Finally, add the aggregate median
stmt3 = select(GameResult.cup, GameResult.mapName).distinct()\
     .order_by(GameResult.cup, GameResult.mapName)

for irow, row in enumerate(session.execute(stmt)):
    this_cup = row[0]
    this_mapName = row[1]
    # Pass to the function that determines median number of generations for this cup-map combo, and inserts it
    add_aggregate_map_points(this_cup, this_mapName)

In [12]:
if session.query(MapPoints).count() > 0:
    print(f"Confirmed that table {MapPoints.__tablename__} is no longer empty")

Confirmed that table sportsbook_stats_points is no longer empty
